In [1]:
# import tweepy
import tweepy as tw

# your Twitter API key and API secret
my_api_key = "ThRPsxUoJDcIsWQFhXLCNz61x"
my_api_secret = "aCiTfjogVc3BfMkNmN8WNvY0uNYZxs8Kjz061cN4F5VXfGtO6L"
my_access_token = "1550320734186385408-xspIru7Tqtoke3Qhecc4T8e6tMGL1S"
my_access_secret = "lUh5lKZpUEauYXleP4blV5Cb0ZavtMuLWJSVz1k4ntIHT"
# authenticate
auth = tw.OAuthHandler(my_api_key, my_api_secret)
auth.set_access_token(my_access_token, my_access_secret)
api = tw.API(auth, wait_on_rate_limit=True)

In [2]:
search_query = "#Rorschach -filter:retweets"


In [3]:
# get tweets from the API
tweets = tw.Cursor(api.search,
              q=search_query,
              lang="en",
              since="2022-07-21").items(50)

# store the API responses in a list
tweets_copy = []
for tweet in tweets:
    tweets_copy.append(tweet)
    
print("Total Tweets fetched:", len(tweets_copy))


Total Tweets fetched: 50


In [4]:
tweets_copy[0].entities['hashtags']

[{'text': 'Rorschach', 'indices': [0, 10]},
 {'text': 'Day5', 'indices': [11, 16]},
 {'text': 'Kochi', 'indices': [65, 71]},
 {'text': 'Mammootty', 'indices': [84, 94]}]

In [5]:
text = api.get_status(id=tweet.id, tweet_mode='extended').full_text
text

'#Rorschach India Net Collection\n\nDay 4: 1.52 Cr\n\nTotal: 10.27 Cr\n\nDetails: https://t.co/uovUz5wDux'

In [6]:
#tweets_no_urls = [remove_url(tweet.text) for tweet in tweets]
#tweet

In [7]:
import pandas as pd
# intialize the dataframe
tweets_df = pd.DataFrame()
# populate the dataframe
for tweet in tweets_copy:
    hashtags = []
    text = ''
    try:
        for hashtag in tweet.entities["hashtags"]:
            hashtags.append(hashtag["text"])
        text = api.get_status(id=tweet.id, tweet_mode='extended').full_text
    except:
        pass
    tweets_df = tweets_df.append(pd.DataFrame({'user_name': tweet.user.name, 
                                               'user_location': tweet.user.location,\
                                               'user_description': tweet.user.description,
                                               'user_verified': tweet.user.verified,
                                               'date': tweet.created_at,
                                               'tweets': text, 
                                               'hashtags': [hashtags if hashtags else None],
                                               'source': tweet.source}))
    tweets_df = tweets_df.reset_index(drop=True)
# show the dataframe
tweets_df.head()

,user_name,user_location,user_description,user_verified,date,tweets,hashtags,source
0,MFWAI KERALA STATE,"Kerala, India",MAMMOOTTY FANS & WELFARE ASSOCIATION INTERNATI...,False,2022-10-11 12:23:35,#Rorschach #Day5 Today's night shows booking s...,"[Rorschach, Day5, Kochi, Mammootty]",Twitter for iPhone
1,S,🇮🇳,Depressed Guy Who Loves Cinema & Memes. 🐿,False,2022-10-11 12:21:52,Brilliant Thread 👌\nThanks for the detailing @...,[Rorschach],Twitter for Android
2,abhi,,You are the master of your thoughts and actions 🌻,False,2022-10-11 12:16:52,#Bheeshmaparvam\n#Cbi5\n#Rorschach \n\n8 films...,"[Bheeshmaparvam, Cbi5, Rorschach, Mammooty]",Twitter for Android
3,Jafar John Kurishinkal,,ഇവിടെ അടുത്താ വീട്.. \nഞങ്ങടെ ഇക്കയെ അറിയും മമ...,False,2022-10-11 12:10:21,#Rorschach Running Successfully with all your ...,"[Rorschach, Mammootty]",Twitter for Android
4,IKKA FANZ CLUB,"Kerala, India",The Twitter Fan page of MegaStar @mammukka ❤️\...,False,2022-10-11 12:10:02,#Rorschach Running Successfully with awesome ...,"[Rorschach, Mammootty, MammoottyKampany]",Twitter for Android


In [8]:
import re

In [9]:
def clean(text):
  cleaned = re.compile(r'<.*?>')
  return re.sub(cleaned,'',text)
tweets_df.tweets = tweets_df.tweets.apply(clean)
tweets_df.tweets[0]

"#Rorschach #Day5 Today's night shows booking status at PVR Lulu, #Kochi🔥\n\n@mammukka #Mammootty https://t.co/vCamin5t3L"

In [10]:
#remove special characters
def is_special(text):
  rem = ''
  for i in text:
    if i.isalnum():
      rem = rem+i
    else:
      rem = rem+" "
  return rem

tweets_df.tweets= tweets_df.tweets.apply(is_special)
tweets_df.tweets[1]

'Brilliant Thread   Thanks for the detailing  urstrulyadhil     Rorschach https   t co 1WdI3qmMq5'

In [11]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

In [12]:
#remove stopwords
nltk.download("stopwords")
nltk.download("punkt")
def rem_stopwords(text):
  stop_words = set(stopwords.words("english"))
  words = word_tokenize(text)
  return  [w for w in words if w not in stop_words]

tweets_df.tweets = tweets_df.tweets.apply(rem_stopwords)
tweets_df.tweets[0]

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


['Rorschach',
 'Day5',
 'Today',
 'night',
 'shows',
 'booking',
 'status',
 'PVR',
 'Lulu',
 'Kochi',
 'mammukka',
 'Mammootty',
 'https',
 'co',
 'vCamin5t3L']

In [13]:
from nltk.stem.wordnet import WordNetLemmatizer
lemmatizing = WordNetLemmatizer()
import nltk
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


True

In [14]:
tweets_df['tweet_lemmatized'] = tweets_df['tweets'].apply(lambda x: ' '.join([lemmatizing.lemmatize(i) for i in x]))
tweets_df['tweet_lemmatized'][0]

'Rorschach Day5 Today night show booking status PVR Lulu Kochi mammukka Mammootty http co vCamin5t3L'

In [15]:
tweets_df.tweets[0]

['Rorschach',
 'Day5',
 'Today',
 'night',
 'shows',
 'booking',
 'status',
 'PVR',
 'Lulu',
 'Kochi',
 'mammukka',
 'Mammootty',
 'https',
 'co',
 'vCamin5t3L']

In [16]:
from textblob import TextBlob

#create a function to get the subjectivity

def getSubjectivity(text):
  return TextBlob(text).sentiment.subjectivity

#create  a function to get the polarity

def getPolarity(text):
  return TextBlob(text).sentiment.polarity

#create two new columns
tweets_df['Subjectivity'] = tweets_df['tweet_lemmatized'].apply(getSubjectivity)
tweets_df['Polarity'] = tweets_df['tweet_lemmatized'].apply(getPolarity)
tweets_df


,user_name,user_location,user_description,user_verified,date,tweets,hashtags,source,tweet_lemmatized,Subjectivity,Polarity
0,MFWAI KERALA STATE,"Kerala, India",MAMMOOTTY FANS & WELFARE ASSOCIATION INTERNATI...,False,2022-10-11 12:23:35,"[Rorschach, Day5, Today, night, shows, booking...","[Rorschach, Day5, Kochi, Mammootty]",Twitter for iPhone,Rorschach Day5 Today night show booking status...,0.000000,0.000000
1,S,🇮🇳,Depressed Guy Who Loves Cinema & Memes. 🐿,False,2022-10-11 12:21:52,"[Brilliant, Thread, Thanks, detailing, urstrul...",[Rorschach],Twitter for Android,Brilliant Thread Thanks detailing urstrulyadhi...,0.600000,0.550000
2,abhi,,You are the master of your thoughts and actions 🌻,False,2022-10-11 12:16:52,"[Bheeshmaparvam, Cbi5, Rorschach, 8, films, ac...","[Bheeshmaparvam, Cbi5, Rorschach, Mammooty]",Twitter for Android,Bheeshmaparvam Cbi5 Rorschach 8 film achieved ...,0.000000,0.000000
3,Jafar John Kurishinkal,,ഇവിടെ അടുത്താ വീട്.. \nഞങ്ങടെ ഇക്കയെ അറിയും മമ...,False,2022-10-11 12:10:21,"[Rorschach, Running, Successfully, love, Luke,...","[Rorschach, Mammootty]",Twitter for Android,Rorschach Running Successfully love Luke Anton...,0.600000,0.500000
4,IKKA FANZ CLUB,"Kerala, India",The Twitter Fan page of MegaStar @mammukka ❤️\...,False,2022-10-11 12:10:02,"[Rorschach, Running, Successfully, awesome, re...","[Rorschach, Mammootty, MammoottyKampany]",Twitter for Android,Rorschach Running Successfully awesome report ...,1.000000,1.000000
5,xavi,,Cinema 🎥❤️,False,2022-10-11 12:09:20,"[Bheeshmaparvam, Cbi5, Rorschach, 8, films, ac...",None,Twitter for Android,Bheeshmaparvam Cbi5 Rorschach 8 film achieved ...,0.000000,0.000000
6,Tollywood,Hyderabad,"All about Tollywood Movie Updates, Lyrics, Mov...",False,2022-10-11 11:53:35,"[Rorschach, Digital, Rights, Satellite, Rights...",[Rorschach],Postcron App,Rorschach Digital Rights Satellite Rights OTT ...,0.187500,-0.062500
7,Team Mannadiyar Online Editors,,𝙾𝚏𝚏𝚒𝚌𝚒𝚊𝚕 𝚃𝚠𝚒𝚝𝚝𝚎𝚛 𝙸𝙳 𝙵𝚘𝚛 𝚃𝙴𝙰𝙼 𝙼𝙰𝙽𝙽𝙰𝙳𝙸𝚈𝙰𝚁 𝙾𝙽𝙻𝙸𝙽𝙴...,False,2022-10-11 11:53:26,"[Rorschach, Running, Successfully, love, mammu...","[Rorschach, MammoottyKampany]",Twitter for Android,Rorschach Running Successfully love mammukka M...,0.600000,0.500000
8,King Always King 👑,,Annum innum ennum #MegastarMammookka uyir 💖💖💖💖...,False,2022-10-11 11:44:08,"[Kottayam, Evening, Night, shows, Mammootty, R...","[Mammootty, Rorschach]",Twitter for Android,Kottayam Evening Night show Mammootty Rorschac...,0.000000,0.000000
9,𝘡𝘶𝘧𝘪 ☻︎,"""........ Dreamtime is over, Come back down to...",GodSpeed !!,False,2022-10-11 11:43:46,"[urstrulyadhil, I, think, loops, depth, film, ...",None,Twitter for iPhone,urstrulyadhil I think loop depth film deep If ...,0.637500,0.250000


In [17]:
def getanalysis(score):
  if score < 0:
    return 'Negative'
  elif score == 0 :
      return 'Neutral'
  else:
      return 'Positive'
tweets_df['Analysis'] = tweets_df['Polarity'].apply(getanalysis)

tweets_df

,user_name,user_location,user_description,user_verified,date,tweets,hashtags,source,tweet_lemmatized,Subjectivity,Polarity,Analysis
0,MFWAI KERALA STATE,"Kerala, India",MAMMOOTTY FANS & WELFARE ASSOCIATION INTERNATI...,False,2022-10-11 12:23:35,"[Rorschach, Day5, Today, night, shows, booking...","[Rorschach, Day5, Kochi, Mammootty]",Twitter for iPhone,Rorschach Day5 Today night show booking status...,0.000000,0.000000,Neutral
1,S,🇮🇳,Depressed Guy Who Loves Cinema & Memes. 🐿,False,2022-10-11 12:21:52,"[Brilliant, Thread, Thanks, detailing, urstrul...",[Rorschach],Twitter for Android,Brilliant Thread Thanks detailing urstrulyadhi...,0.600000,0.550000,Positive
2,abhi,,You are the master of your thoughts and actions 🌻,False,2022-10-11 12:16:52,"[Bheeshmaparvam, Cbi5, Rorschach, 8, films, ac...","[Bheeshmaparvam, Cbi5, Rorschach, Mammooty]",Twitter for Android,Bheeshmaparvam Cbi5 Rorschach 8 film achieved ...,0.000000,0.000000,Neutral
3,Jafar John Kurishinkal,,ഇവിടെ അടുത്താ വീട്.. \nഞങ്ങടെ ഇക്കയെ അറിയും മമ...,False,2022-10-11 12:10:21,"[Rorschach, Running, Successfully, love, Luke,...","[Rorschach, Mammootty]",Twitter for Android,Rorschach Running Successfully love Luke Anton...,0.600000,0.500000,Positive
4,IKKA FANZ CLUB,"Kerala, India",The Twitter Fan page of MegaStar @mammukka ❤️\...,False,2022-10-11 12:10:02,"[Rorschach, Running, Successfully, awesome, re...","[Rorschach, Mammootty, MammoottyKampany]",Twitter for Android,Rorschach Running Successfully awesome report ...,1.000000,1.000000,Positive
5,xavi,,Cinema 🎥❤️,False,2022-10-11 12:09:20,"[Bheeshmaparvam, Cbi5, Rorschach, 8, films, ac...",None,Twitter for Android,Bheeshmaparvam Cbi5 Rorschach 8 film achieved ...,0.000000,0.000000,Neutral
6,Tollywood,Hyderabad,"All about Tollywood Movie Updates, Lyrics, Mov...",False,2022-10-11 11:53:35,"[Rorschach, Digital, Rights, Satellite, Rights...",[Rorschach],Postcron App,Rorschach Digital Rights Satellite Rights OTT ...,0.187500,-0.062500,Negative
7,Team Mannadiyar Online Editors,,𝙾𝚏𝚏𝚒𝚌𝚒𝚊𝚕 𝚃𝚠𝚒𝚝𝚝𝚎𝚛 𝙸𝙳 𝙵𝚘𝚛 𝚃𝙴𝙰𝙼 𝙼𝙰𝙽𝙽𝙰𝙳𝙸𝚈𝙰𝚁 𝙾𝙽𝙻𝙸𝙽𝙴...,False,2022-10-11 11:53:26,"[Rorschach, Running, Successfully, love, mammu...","[Rorschach, MammoottyKampany]",Twitter for Android,Rorschach Running Successfully love mammukka M...,0.600000,0.500000,Positive
8,King Always King 👑,,Annum innum ennum #MegastarMammookka uyir 💖💖💖💖...,False,2022-10-11 11:44:08,"[Kottayam, Evening, Night, shows, Mammootty, R...","[Mammootty, Rorschach]",Twitter for Android,Kottayam Evening Night show Mammootty Rorschac...,0.000000,0.000000,Neutral
9,𝘡𝘶𝘧𝘪 ☻︎,"""........ Dreamtime is over, Come back down to...",GodSpeed !!,False,2022-10-11 11:43:46,"[urstrulyadhil, I, think, loops, depth, film, ...",None,Twitter for iPhone,urstrulyadhil I think loop depth film deep If ...,0.637500,0.250000,Positive


In [18]:
ptweets = tweets_df[tweets_df.Analysis =='Positive']
ptweets = ptweets['tweet_lemmatized']
 
round ( (ptweets.shape[0]/tweets_df.shape[0]) *100,1) 

68.0

In [19]:
ntweets = tweets_df[tweets_df.Analysis =='Negative']
ntweets = ntweets['tweet_lemmatized']
 
round ( (ntweets.shape[0]/tweets_df.shape[0]) *100,1)

4.0